In [1]:
import dspy
import anaconda_ai.integrations.litellm  # noqa: F401

In [2]:
lm = dspy.LM(
    "anaconda/TinyLlama-1.1B-Chat-v1.0_Q4_K_M.gguf",
    cache=False,
    temperature=0.1,
    optional_params={"server": {"ctx_size": 512}},
)
dspy.configure(lm=lm)

In [3]:
res = lm("what is pi?")
print(res[0])

Output()

✓ TinyLlama-1.1B-Chat-v1.0_Q4_K_M.gguf (running)


pi is a mathematical constant that represents the ratio of the circumference of a circle to its diameter. It is defined as 3.141592653589793238462643383279502884197169399375105820974944592307816406286208998628034825342105559572704259035837668939609375.


In [4]:
fact_checking = dspy.asyncify(dspy.ChainOfThought("claims -> verdicts: list[bool]"))

res = await fact_checking(
    claims=["Python was released in 1991.", "Python is a compiled language."]
)
res

Prediction(
    reasoning="['Python was released in 1991.']",
    verdicts=[True, False]
)

In [5]:
embedder = dspy.Embedder("anaconda/bge-small-en-v1.5/q4_k_m", batch_size=100)
embeddings = embedder(["hello", "world"])

In [6]:
embeddings.shape

(2, 384)

In [7]:
output = await embedder.acall(["cat"])
output.shape

(1, 384)

Output()